In [1]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR

C:\Users\kiran kumar\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kiran kumar\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kiran kumar\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
 
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = pd.read_csv(url, header=None)

In [3]:
dataframe

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [5]:

# split into inputs and outputs
last_x = len(dataframe.columns) - 1
X, y = dataframe.drop(last_x, axis=1), dataframe[last_x]
print(X.shape, y.shape)

(4177, 8) (4177,)


In [6]:
X.dtypes

0     object
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
dtype: object

In [7]:

# determine categorical and numerical features
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

In [24]:

# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', StandardScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)
col_transform

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=True),
                                 Int64Index([0], dtype='int64')),
                                ('num',
                                 StandardScaler(copy=True, with_mean=True,
                                                with_std=True),
                                 Int64Index([1, 2, 3, 4, 5, 6, 7], dtype='int64'))],
                  verbose=False)

In [19]:
kiran = pd.DataFrame(col_transform.fit_transform(X))
kiran

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,1.0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217
1,0.0,0.0,1.0,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987
2,1.0,0.0,0.0,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139
3,0.0,0.0,1.0,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294
4,0.0,1.0,0.0,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757
...,...,...,...,...,...,...,...,...,...,...
4172,1.0,0.0,0.0,0.341509,0.424464,0.609334,0.118813,0.047908,0.532900,0.073062
4173,0.0,0.0,1.0,0.549706,0.323686,-0.107991,0.279929,0.358808,0.309362,0.155685
4174,0.0,0.0,1.0,0.632985,0.676409,1.565767,0.708212,0.748559,0.975413,0.496955
4175,1.0,0.0,0.0,0.841182,0.777187,0.250672,0.541998,0.773341,0.733627,0.410739


In [20]:

# define the model
model = SVR(kernel='rbf',gamma='scale',C=100)

# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])

# define the model cross-validation configuration
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# evaluate the pipeline using cross validation and calculate MAE
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# convert MAE scores to positive values
scores = absolute(scores)
# summarize the model performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 1.491 (0.047)
